# Simple Example of using WebLoader with Keras

In [1]:
import os
import logging
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras import models, layers
import numpy as np

from keras.utils.training_utils import multi_gpu_model
import numpy as np
import webloader as wl

from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [2]:
#from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

In [3]:
#!test -f imagenet_val-0000-000.tgz || curl http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_val-0000-000.tgz -o imagenet_val-0000.tgz

In [4]:
num_gpus = 1

if True:
    # user smaller epochs for the demo
    training_size = 10000
    test_size = 5000
else:
    # true epoch sizes
    training_size = 1000000
    test_size = 5000

urls = "http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_train-{0000..0147}-{000..019}.tgz"
batch_size = 64
num_batches = training_size//batch_size

test_urls = "http://storage.googleapis.com/lpr-imagenet-augmented/imagenet_val-0000-000.tgz"
test_batch_size = 256
test_num_batches = test_size//test_batch_size

In [5]:
mean = np.array([0.485, 0.456, 0.406], "f")
std = np.array([0.229, 0.224, 0.225], "f")
def norm_image(xs):
    return (xs-mean[None,None,None,:])/std[None,None,None,:]
def norm_cls(ys):
    return keras.utils.to_categorical(ys-1, 1000)

In [6]:
testing = wl.WebLoader(test_urls, test_size,
                       fields="ppm;png;jpg cls",
                       shuffle=1000,
                       batch_transforms=[norm_image, norm_cls],
                       batch_size=test_batch_size,
                       epochs=-1)
wl.loader_test(testing)

263.10 samples/s 1.03 batches/s
Example:
0 : ndarray (256, 224, 224, 3) float32 -2.117904 2.64
1 : ndarray (256, 1000) float32 0.0 1.0


In [7]:
# you can use a WebLoader here and multiprocessing in model.fit_generator,
# or you can use a MultiWebLoader here and disable multiprocessing in model.fit_generator
training = wl.MultiWebLoader(urls, training_size,
                       fields="ppm;png;jpg cls",
                       batch_transforms=[norm_image, norm_cls],
                       batch_size=batch_size,
                       processes=4,
                       shuffle=1000)
wl.loader_test(training)

461.41 samples/s 7.21 batches/s
Example:
0 : ndarray (64, 224, 224, 3) float32 -2.117904 2.64
1 : ndarray (64, 1000) float32 0.0 1.0


In [8]:
lr = 0.001 # was: 0.0001

model = ResNet50()

if num_gpus>1: model = multi_gpu_model(model, gpus=num_gpus)

opt = keras.optimizers.rmsprop(lr=lr, decay=1e-6)

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit_generator(iter(training),
              epochs=3,
              steps_per_epoch=num_batches, #num_batches,
              validation_data=iter(testing),
              validation_steps=test_num_batches, #test_num_batches,
              shuffle=False,
              use_multiprocessing=False,
              workers=1)

Epoch 1/3
156/156 [==============================] - 69s 444ms/step - loss: 7.1328 - acc: 0.0017 - val_loss: 9.2291 - val_acc: 0.0016
Epoch 2/3
156/156 [==============================] - 61s 391ms/step - loss: 6.8958 - acc: 0.0027 - val_loss: 9.6708 - val_acc: 0.0014
Epoch 3/3
156/156 [==============================] - 62s 395ms/step - loss: 6.8141 - acc: 0.0035 - val_loss: 7.0145 - val_acc: 0.0029
